In [5]:
from bson.code import Code
from bson.son import SON
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.recommandation_db

In [6]:
mapper=Code('''
    function(){
        emit(this.userId,this.movieId);
    };
''')
reducer=Code('''
    function(key,values){
        var arr=[];
        for(var i=0;i<values.length;i++){
            arr.push(values[i]);
        }
        return arr;
    };
''')
db.train_set.map_reduce(mapper,reducer,'train_user_movies')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'train_user_movies')

In [7]:
def recommendation_score(I):
    cur=db.train_user_movies.aggregate([ 
        {'$project':{
            'userId':'$_id',
            'movies':'$value',
            '_id':0
        }},
        {"$out": "temp_train_set"},
    ])
#     -------------------------------------------------------------------------------
    user_i=db.temp_train_set.find_one({'userId':I})
    user_i_movielist=user_i['movies']
    
    db.temp_train_set.update_many(
        {},
        {'$set':
            {
                'user_i_movielist':user_i_movielist
            }
        },
    )
#     -------------------------------------------------------------------------------
    cur=db.temp_train_set.aggregate([
        {
            '$project':{
                'userId':1,
                'user_i_movielist':1,
                'movies':1,
                'commonMovieIDs':{'$setIntersection':['$movies','$user_i_movielist']},
                'unionMovieIDs':{'$setUnion':['$movies','$user_i_movielist']},
            }
        },
        {
            '$project':{
                'userId':1,
                'user_i_movielist':1,
                'movies':1,
                'commonMovieIDs':1,
                'diffMovieIDs':{'$setDifference':['$unionMovieIDs','$commonMovieIDs']},
                's_ij':{'$size':'$commonMovieIDs'}
            }
        },
        {
            '$project':{
                'userId':1,
                'user_i_movielist':1,
                'movies':1,
                'commonMovieIDs':1,
                'diffMovieIDs':1,
                's_ij':1,
                'd_ij':{'$size':'$diffMovieIDs'}
            }
        },
        {
            '$group':{
                '_id':'',
                'sum_d_ij':{'$sum':'$d_ij'},
                'sum_s_ij':{'$sum':'$s_ij'},
            }
        }
    ])
#     -------------------------------------------------------------------------------
    a=list(cur)
    sum_d_ij=a[0]['sum_d_ij']
    sum_s_ij=a[0]['sum_s_ij']
#     -------------------------------------------------------------------------------
    cur=db.temp_train_set.aggregate([
        {
            '$project':{
                'userId':1,
                'user_i_movielist':1,
                'movies':1,
                'commonMovieIDs':{'$setIntersection':['$movies','$user_i_movielist']},
                'unionMovieIDs':{'$setUnion':['$movies','$user_i_movielist']},
            }
        },
        {
            '$project':{
                'userId':1,
                'user_i_movielist':1,
                'movies':1,
                'commonMovieIDs':1,
                'diffMovieIDs':{'$setDifference':['$unionMovieIDs','$commonMovieIDs']},
                's_ij':{'$size':'$commonMovieIDs'}
            }
        },
        {
            '$project':{
                'userId':1,
                'movies':1,
                's_ij':1,
                'd_ij':{'$size':'$diffMovieIDs'}
            }
        },
        {
            '$project':{
                'userId':1,
                'movies':1,
                'hat_d_ij':{'$divide':['$d_ij',sum_d_ij]},
                'hat_s_ij':{'$divide':['$s_ij',sum_s_ij]},
                '_id':0
                }
            },
        {'$unwind':'$movies'},
        {'$out':'temp_collection'}
    ])
#     -------------------------------------------------------------------------------
    cur=db.temp_collection.aggregate([
        {
            '$group':{
                '_id':'$movies',
                'positive_score':{'$sum':'$hat_s_ij'}
            }
        },
        {'$out':'temp_positive'}
    ])
#     -------------------------------------------------------------------------------
    cur=db.temp_collection.aggregate([
        {
            '$group':{
                '_id':'$movies',
                'negative_score':{'$sum':'$hat_d_ij'}
            }
        },
        {
            '$lookup':{
                'from':'temp_positive',
                'localField':'_id',
                'foreignField':'_id',
                'as':'positive_score'
            }
        },
        {
            '$project':{
                'movieId':'$_id',
                'negative_score':'$negative_score',
                'positive_score':'$positive_score.positive_score',
                '_id':0
            }
        },
        {'$unwind':'$positive_score'},
        {
            '$addFields':{
                'user_I':I
            }
        },
        {
            '$merge':{
                'into':'recommendation_scores',
                'on':'_id',
                'whenMatched':'keepExisting',
                'whenNotMatched':'insert'
            }
        }
    ])
#     -------------------------------------------------------------------------------
    db.temp_train_set.drop()
    db.temp_collection.drop()
    db.temp_positive.drop()


In [8]:
cur=db.train_user_movies.aggregate([
    {'$sort':{
        '_id':1
    }},
    {
        '$project':{
            '_id':1
        }
    }
])
user_list=list(cur)

In [9]:
for i in range(0,2000):
    recommendation_score(user_list[i]['_id'])